In [1]:
from dotenv import load_dotenv
load_dotenv()

from ecmwf.opendata import Client
from datetime import timedelta
import pandas as pd
import time
import logging
from pymongo import MongoClient

from pipeline.downloader import Downloader
from pipeline.orchestrator import PipelineOrchestrator
from pipeline.runner import PipelineRunner
from db.repo import ResultsRepository
from db.mongo import MongoWrapper

from pipeline.config import settings

from datetime import datetime

from model.forecast import base_vs_forecast, forecast_vs_forecast, full_forecast

In [2]:
steps = [i for i in range(0, 144, 3)] + [j for j in range(144, 361, 6)] #get all steps
var = '2t' #forecast returned variables

request = {
    'date': -1,
    'time': 12,
    'type': "fc",
    'stream': 'oper',
    'step': steps,
    'param': var,
}    
d = Downloader(request=request, download_dir=settings.DOWNLOAD_DIR)
filepath = d.download()

To ensure the stability of our systems and to preserve resources for our operational activities (network, compute, etc.), access to the open-data portal is limited to 500 simultaneous connections. This limit helps us guarantee reliable service for our operational users, especially during periods of high demand. For added reliability, the open-data is replicated across AWS, Azure, and Google Cloud. If you experience difficulties accessing the portal directly, you can also retrieve the data from these cloud platforms.
                                                                

By downloading data from the ECMWF open data dataset, you agree to the terms: Attribution 4.0 International (CC BY 4.0). Please attribute ECMWF when downloading this data.


In [3]:
latest_run_date = d.check_latest_available()
previous_run_date = pd.Timestamp(latest_run_date) - pd.Timedelta(hours=12)

In [4]:
horizons = [(previous_run_date.date() + timedelta(days=1), previous_run_date.date() + timedelta(days=3), 'Day 1-3'),
                    (previous_run_date.date() + timedelta(days=4), previous_run_date.date() + timedelta(days=7), 'Day 4-7'),
                    (previous_run_date.date() + timedelta(days=8), previous_run_date.date() + timedelta(days=14), 'Day 8-14')]

base_forecast = base_vs_forecast(filepath, settings.BASE_FILE_PATH, horizons)

Ignoring index file 'data/current.grib2.5b7b6.idx' older than GRIB file


In [7]:
mongo = MongoWrapper(settings.MONGO_URI, settings.MONGO_DB)
repo = ResultsRepository(mongo_client=mongo, repo=settings.MONGO_COLLECTION)

repo.insert_results(base_forecast)